In [1]:
%%HTML
<style>.container{width:100%}</style>

In [11]:
def id2bit(ls: list):
    """
    Converts a list of indices into a binary representation (bit vector).

    Given a list of indices (ls), this function returns a list of bits where
    the bit positions corresponding to the indices in the list are set to 1,
    and all other positions are set to 0. The resulting list is reversed.

    Args:
        ls (list): A list of indices to be converted to bits.

    Returns:
        list: A list of bits representing the binary values.
    """
    if len(ls) == 0:
        return [0, 0, 0, 0, 0, 0, 0, 0]  # Return a default 8-bit array
    aa = [0 for _ in range(max(ls) + 1)]
    for i in ls:
        aa[i] = 1
    return aa[::-1]


def bit2id(ls: list, log=False):
    """
    Converts a binary list (bit vector) back to a list of indices.

    Given a list of bits (ls), this function returns the indices of the bits
    that are set to 1. The binary list is reversed during the conversion.

    Args:
        ls (list): A list of bits representing a binary value.
        log (bool, optional): Whether to log intermediate steps (default is False).

    Returns:
        list: A list of indices where the bits are set to 1.
    """
    ls = ls[::-1]
    aa = []

    for i in range(len(ls)):
        if ls[i] == 1:
            aa.append(i)
    return aa[::-1]


def bit2mul(a, b, log=False):
    """
    Multiplies two binary numbers represented as lists of bits.

    This function multiplies two binary numbers by performing a bitwise
    multiplication and addition over Galois Field (GF(2)).

    Args:
        a (list): A list of bits representing the first binary number.
        b (list): A list of bits representing the second binary number.
        log (bool, optional): Whether to log intermediate steps (default is False).

    Returns:
        list: The resulting binary number (list of bits).
    """
    ai = bit2id(a)
    bi = bit2id(b)
    a, b = a[::-1], b[::-1]

    if ai == []:
        return a
    elif bi == []:
        return b

    addn = [[ai[i] + bi[j] for j in range(len(bi))][::-1] for i in range(len(ai))][::-1]
    addn = [id2bit(i) for i in addn]

    maxsiz = max([len(i) for i in addn])
    for i in range(len(addn)):
        if len(addn[i]) < maxsiz:
            addn[i] = [0 for _ in range(maxsiz - len(addn[i]))] + addn[i]

    smm = []
    for i in range(maxsiz):
        t = 0
        for j in addn:
            t += j[i]
        smm.append(t % 2)

    return smm


def bit2add(a, b):
    """
    Adds two binary numbers represented as lists of bits (bitwise addition).

    This function adds two binary numbers by performing a bitwise addition over GF(2).

    Args:
        a (list): A list of bits representing the first binary number.
        b (list): A list of bits representing the second binary number.

    Returns:
        list: The resulting binary number after addition (list of bits).
    """
    a, b = list(a), list(b)
    a, b = a[::-1], b[::-1]
    maxsiz = max(len(a), len(b))

    if len(a) < maxsiz:
        a = a + [0 for _ in range(maxsiz - len(a))]
    if len(b) < maxsiz:
        b = b + [0 for _ in range(maxsiz - len(b))]

    smm = []
    for i in range(maxsiz):
        smm.append((a[i] + b[i]) % 2)

    return smm[::-1]


def bit2str(bit: list):
    """
    Converts a list of bits into a string.

    This function converts a list of binary bits (0s and 1s) into a string of characters.

    Args:
        bit (list): A list of bits (0s and 1s).

    Returns:
        str: The string representation of the binary bits.
    """
    s = ""
    for i in bit:
        s += str(i)
    return s


def str2bit(s: str):
    """
    Converts a string of '0's and '1's into a list of bits.

    This function converts a string containing '0's and '1's into a list of integer bits.

    Args:
        s (str): A string containing '0's and '1's.

    Returns:
        list: A list of bits (integers).

    Raises:
        ValueError: If the string contains characters other than '0' and '1'.
    """
    if set(s).issubset(set("01")):
        bit = [int(i) for i in s]
        return bit
    else:
        print("bit string should contain 1s and 0s")


def modgf(dsr: list, dnt=[1, 0, 0, 0, 1, 1, 0, 1, 1]):
    """
    Performs polynomial division over Galois Field (GF(2)).

    This function divides the binary polynomial `dsr` by the binary polynomial `dnt`
    and returns the quotient and remainder.

    Args:
        dsr (list): The dividend as a list of bits (binary polynomial).
        dnt (list, optional): The divisor as a list of bits (default is a predefined irreducible polynomial).

    Returns:
        tuple: The remainder and quotient as lists of bits.
    """
    dsr = bit2id(dsr)
    dnt = bit2id(dnt)
    qtnt = []

    while len(dnt) != 0 and len(dsr) != 0 and (max(dnt) - max(dsr) >= 0):
        ml = max(dnt) - max(dsr)
        qtnt.append(ml)
        plus = id2bit(dnt)
        minus = id2bit([ml + i for i in dsr])
        rem = bit2add(plus, minus)
        dnt = bit2id(rem)

    return id2bit(dnt), id2bit(qtnt)


def ext_eucld(a, b, log=False):
    """
    Extended Euclidean algorithm for binary polynomials.

    This function computes the extended Euclidean algorithm for binary polynomials `a` and `b`,
    returning the coefficients of the linear combination of `a` and `b` that equals the greatest common divisor (GCD).

    Args:
        a (list): A list of bits representing the first binary polynomial.
        b (list): A list of bits representing the second binary polynomial.
        log (bool, optional): Whether to log intermediate steps (default is False).

    Returns:
        list: The coefficients of the linear combination of `a` and `b` (as lists of bits).
    """
    ai, bi = bit2id(a), bit2id(b)
    if len(ai) != 0 and len(bi) != 0:
        if max(max(ai), max(bi)) == max(bi):
            a, b = b, a
    elif len(ai) == 0 and len(bi) != 0:
        a, b = b, a

    def eucld(a, b, log=False):
        a, b = a[::-1], b[::-1]

        if set(b) == set([0]) or (b[0] == 1 and (set(b[1:]) == set([0]))):
            return []

        ls = []

        while not (b[0] == 1 and (set(b[1:]) == set([0]))):
            r, idx = modgf(b[::-1], dnt=a[::-1])
            r, idx = r[::-1], idx[::-1]

            if set(r) == set([0]):
                return ls

            ls.append(idx[::-1])
            a = b
            b = r
        return ls

    row = [
        [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0]],
        [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1]],
    ]

    ls = eucld(a, b)
    for i in ls:
        r10, r11 = row[-1][0], row[-1][1]
        r20, r21 = row[-2][0], row[-2][1]
        r0 = bit2add(r20, bit2mul(r10, i))
        r1 = bit2add(r21, bit2mul(r11, i))
        rowl = [r0, r1]
        row.append(rowl)

    return row[-1]


def Gfinv(bit, irrpoly=[1, 0, 0, 0, 1, 1, 0, 1, 1]):
    """
    Computes the multiplicative inverse of a binary polynomial over GF(2).

    This function uses the extended Euclidean algorithm to compute the inverse of a binary polynomial `bit`
    with respect to a predefined irreducible polynomial `irrpoly`.

    Args:
        bit (list): A list of bits representing the binary polynomial to be inverted.
        irrpoly (list, optional): The irreducible polynomial used for the field (default is a predefined polynomial).

    Returns:
        list: The multiplicative inverse of the polynomial `bit` (list of bits).
    """
    if set(bit) == set("0"):
        return "--"

    ans = ext_eucld(irrpoly, bit)
    ans = ans[-1][-len(bit) :]
    return ans


# Example call
Gfinv([0, 0, 0, 0, 0, 1, 0, 0], irrpoly=[0, 0, 0, 1, 0, 0, 1, 1])

[0, 0, 0, 0, 1, 1, 0, 1]

In [12]:
def genmapping(n: int, irrpoly):
    """
    Generates the elements of GF(2^n) and their corresponding multiplicative inverses
    based on the provided irreducible polynomial.

    Parameters:
    n (int): The size of the Galois Field (GF(2^n)). Determines the number of elements
             in the field, which is 2^n.
    irrpoly (list): A list of bits representing the irreducible polynomial used
                    for the finite field operations (e.g., [1, 0, 0, 1] for x^3 + 1).

    Returns:
    tuple: A tuple containing:
        - gf (list): A list of binary strings of length `n`, representing all elements
                     of GF(2^n). The binary strings are padded with leading zeros.
        - invmap (dict): A dictionary mapping the index of each element in `gf` to the
                         index of its multiplicative inverse, using the irreducible
                         polynomial for the field.

    Example:
    gf, invmap = genmapping(3, [1, 0, 0, 1])
    # gf will contain the elements ['000', '001', '010', '011', '100', '101', '110', '111']
    # invmap will contain a mapping of the inverses for each non-zero element.
    """
    gf = [str(bin(i))[2:] for i in range(2**n)]

    # Ensure each element has length n (pad with leading zeros if necessary)
    for i in range(len(gf)):
        if len(gf[i]) < n:
            gf[i] = "0" * (n - len(gf[i])) + gf[i]

    # Create mappings: index -> element (key2ele) and element -> index (ele2key)
    key2ele = dict(enumerate(gf))
    ele2key = dict([i[::-1] for i in list(enumerate(gf))])

    # Generate the inverse map for all non-zero elements
    invmap = dict()
    for i in gf:
        if set(i) != set("0"):  # Skip zero element
            inv = bit2str(Gfinv(str2bit(i), irrpoly=irrpoly))  # Find the inverse of i
            invmap[ele2key[i]] = ele2key[
                inv
            ]  # Map the inverse using element-to-key mapping

    return gf, invmap

In [13]:
gf5, invmap = genmapping(n=5, irrpoly=id2bit([5, 2, 0]))

In [14]:
set(invmap.values()) == set(invmap.keys())

True

In [15]:
invmap

{1: 1,
 2: 18,
 3: 28,
 4: 9,
 5: 23,
 6: 14,
 7: 12,
 8: 22,
 9: 4,
 10: 25,
 11: 16,
 12: 7,
 13: 15,
 14: 6,
 15: 13,
 16: 11,
 17: 24,
 18: 2,
 19: 29,
 20: 30,
 21: 26,
 22: 8,
 23: 5,
 24: 17,
 25: 10,
 26: 21,
 27: 31,
 28: 3,
 29: 19,
 30: 20,
 31: 27}

In [9]:
gf28 = [str(bin(i))[2:] for i in range(256)]
for i in range(len(gf28)):
    if len(gf28[i]) < 8:
        gf28[i] = "0" * (8 - len(gf28[i])) + gf28[i]

key2ele = dict(enumerate(gf28))
ele2key = dict([i[::-1] for i in list(enumerate(gf28))])
invmap = dict()
for i in gf28:
    if set(i) != set("0"):
        inv = bit2str(Gfinv(str2bit(i)))
        invmap[ele2key[i]] = ele2key[inv]

In [10]:
set(invmap.values()) == set(invmap.keys())

True